In [21]:
# Загрузим нужные библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pylab import plt, mpl
import seaborn as sns
mpl.rcParams['font.family'] = 'serif'
import math
%matplotlib inline
plt.style.use('fivethirtyeight')
# Чистка отклонений
import scipy.stats as stats
# Уберем warnings
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
# Тематическое моделирование
import gensim

from scipy import sparse
import xgboost as xgb
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from lightgbm import LGBMClassifier
import lightgbm as lgbm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold

# Feature Engineering

In [13]:



train_df = pd.read_json('train.json')
test_df = pd.read_json('test.json')




# Чистка широты и долготы
train_df["longitude"]=train_df["longitude"].apply(lambda x:-73.75 if x>=-73.75 else x)
test_df["longitude"]=test_df["longitude"].apply(lambda x:-74.05 if x<=-74.05 else x)
train_df["latitude"]=train_df["latitude"].apply(lambda x:40.4 if x<=40.4 else x)
test_df["latitude"]=test_df["latitude"].apply(lambda x:40.9 if x>=40.9 else x)

mean_price = int(train_df['price'].mean())
test_df.loc[test_df['price']<200,'price'] = mean_price
train_df.loc[train_df['price']<200,'price'] = mean_price
# Метапризнак предсказ цены на основе координат
#X_train_coor = train_df[['latitude', 'longitude']]
#X_test_coor = test_df[['latitude', 'longitude']]

#y_train_price = train_df.price
#y_test_price = test_df.price

#neigh_train = KNeighborsRegressor(n_neighbors=2)
#neigh_train.fit(X_train_coor, y_train_price)

#neigh_test = KNeighborsRegressor(n_neighbors=2)
#neigh_test.fit(X_test_coor, y_test_price)

#train_df['price_comparison'] = train_df['price'] /  neigh_train.predict(X_train_coor)
#test_df['price_comparison'] = test_df['price'] /  neigh_test.predict(X_test_coor)


# Функция расчета расстояния от манхэттэна
def distance_pair(lat1, lon1, lat2, lon2):
    # Перевод в радианы
    p = 0.017453292519943295     #Pi/180
    haver_formula = 0.5 - np.cos((lat2 - lat1) * p)/2 +  np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    d_2_point = 6367 *2 * np.arcsin(np.sqrt(haver_formula)) 
    return d_2_point
distance_pairs = np.vectorize(distance_pair)

# Расстояние от финансового центра
train_df['distance_manh'] = distance_pairs(train_df.latitude, train_df.longitude, 40.70858733058446, -74.01098113951349)
test_df['distance_manh'] = distance_pairs(test_df.latitude, test_df.longitude, 40.70858733058446, -74.01098113951349)


# Расстояние от центр парка
train_df['distance_central_park'] = distance_pairs(train_df.latitude, train_df.longitude, 40.77897128595648, -73.96656147253005)
test_df['distance_central_park'] = distance_pairs(test_df.latitude, test_df.longitude, 40.77897128595648, -73.96656147253005)

# Расстояние от  Бронкс
train_df['bronx'] = distance_pairs(train_df.latitude, train_df.longitude, 40.84928609864644, -73.88844362698887)
test_df['bronx'] = distance_pairs(test_df.latitude, test_df.longitude, 40.84928609864644, -73.88844362698887)

# Расстояние от Район Queens

train_df['queens'] = distance_pairs(train_df.latitude, train_df.longitude, 40.747803602867066, -73.90170458998642)
test_df['queens'] = distance_pairs(test_df.latitude, test_df.longitude, 40.747803602867066, -73.90170458998642)

# Расстояние от Район Brooklin кладбище

train_df['brooklin_cometery'] = distance_pairs(train_df.latitude, train_df.longitude, 40.68822862016868, -73.87615328604328)
test_df['brooklin_cometery'] = distance_pairs(test_df.latitude, test_df.longitude, 40.68822862016868, -73.87615328604328)

# Расстояние от Район Brooklin
train_df['brooklin'] = distance_pairs(train_df.latitude, train_df.longitude, 40.62506864788317, -73.96786745767038)
test_df['brooklin'] = distance_pairs(test_df.latitude, test_df.longitude, 40.62506864788317, -73.96786745767038)






# Кластеризация координат
geo = ['latitude', 'longitude']
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, random_state=42)

# Метки кластеров внесем в переменную clusters
train_df['clusters'] = kmeans.fit(train_df[geo]).labels_
test_df['clusters'] = kmeans.fit(test_df[geo]).labels_


#test_df = test.copy()


train_df['num_photos'] = train_df['photos'].apply(len)
test_df['num_photos'] = test_df['photos'].apply(len)

# Число признаков
train_df['num_features'] = train_df['features'].apply(len)
test_df['num_features'] = test_df['features'].apply(len)

# Количество слов
train_df['num_description_words'] = train_df['description'].apply(lambda x: len(x.split(" ")))
test_df['num_description_words'] = test_df['description'].apply(lambda x: len(x.split(" ")))


# Даты
train_df['created'] = pd.to_datetime(train_df['created'])
test_df['created'] = pd.to_datetime(test_df['created'])

# Извлекаем время
train_df['created_year'] = train_df['created'].dt.year
test_df['created_year'] = test_df['created'].dt.year
train_df['created_month'] = train_df['created'].dt.month
test_df['created_month'] = test_df['created'].dt.month
train_df['created_day'] = train_df['created'].dt.day
test_df['created_day'] = test_df['created'].dt.day
train_df['created_hour'] = train_df['created'].dt.hour
test_df['created_hour'] = test_df['created'].dt.hour



# Фичи по ванной и спальням
# Разл в комнатах
train_df['room_difference'] = train_df['bedrooms'] - train_df['bathrooms']
test_df['room_difference'] = test_df['bedrooms'] - test_df['bathrooms']
# Общее количество
train_df['total_rooms'] = train_df['bedrooms'] + train_df['bathrooms']
test_df['total_rooms'] = test_df['bedrooms'] + test_df['bathrooms']
# Цена 1 добавляется чтобы исключить деление на ноль
train_df['price_per_room'] = train_df['price'] / (train_df['total_rooms'] + 1)
test_df['price_per_room'] = test_df['price'] / (test_df['total_rooms'] + 1)
# Цена за спальню
train_df['price_per_bedroom'] =train_df['price']/(train_df['bedrooms'] + 1)
test_df['price_per_bedroom'] =test_df['price']/(test_df['bedrooms'] +1)

# Цена за ванную
train_df['price_per_bathroom'] =train_df['price']/(train_df['bathrooms'] +1)
test_df['price_per_bathroom'] =test_df['price']/(test_df['bathrooms'] +1)

# Близость к метро
subway = pd.read_csv('NYC_Transit_Subway_Entrance_And_Exit_Data.csv') # сторонние данные
subway = subway[['Station Name', 'Station Latitude', 'Station Longitude']]
subway = subway.groupby(['Station Name']).mean().reset_index(drop=True)


# Функция минимального расстояния от метро
def get_subway_distance(location):
    distances = distance_pairs(location[0], location[1], subway['Station Latitude'], subway['Station Longitude'])    
    return min(distances)

# Расчет минимальной дистанции до метро
train_df['location'] = train_df[['latitude', 'longitude']].values.tolist()
test_df['location'] = test_df[['latitude', 'longitude']].values.tolist()

train_df['subway_distance'] = train_df['location'].apply(get_subway_distance)
test_df['subway_distance'] = test_df['location'].apply(get_subway_distance)


# Плотность по координатам https://www.kaggle.com/code/maheshak04/rh004
train_df["pos"] = train_df.longitude.round(3).astype(str) + '_' + train_df.latitude.round(3).astype(str)
test_df["pos"] = test_df.longitude.round(3).astype(str) + '_' + test_df.latitude.round(3).astype(str)

vals = train_df['pos'].value_counts()
dvals = vals.to_dict()
train_df["density"] = train_df['pos'].apply(lambda x: dvals.get(x, vals.min()))
test_df["density"] = test_df['pos'].apply(lambda x: dvals.get(x, vals.min()))



# Фичи отсюда https://www.kaggle.com/code/slavik0505/kernel43aae13b62
train_df["listing_id1"] = train_df["listing_id"] - 68119576.0
test_df["listing_id1"] =  test_df["listing_id"] - 68119576.0

train_df["total_days"] =   (train_df["created_month"] -4.0)*30 + train_df["created_day"] +  train_df["created_hour"] /25.0
test_df["total_days"] =(test_df["created_month"] -4.0)*30 + test_df["created_day"] +  test_df["created_hour"] /25.0        
train_df["diff_rank"]= train_df["total_days"]/train_df["listing_id1"]
test_df["diff_rank"]= test_df["total_days"]/test_df["listing_id1"]

# Фича из фото нужно скачать торрент файл
df_time = pd.read_csv('listing_image_time.csv')
train_df = train_df.merge(df_time, how='left', on='listing_id', suffixes=('', '_remove'))
test_df = test_df.merge(df_time, how='left', on='listing_id', suffixes=('', '_remove'))

In [14]:
# Фичи отсюда https://www.kaggle.com/code/slavik0505/kernel43aae13b62
train_df["price0"] = (train_df["price"]%10).astype(int)
test_df["price0"] = (test_df["price"]%10).astype(int)

In [15]:
# Фичи отсюда https://www.kaggle.com/code/slavik0505/kernel43aae13b62
train_df["price_latitude"] = (train_df["price"])/ (train_df["latitude"]+1.0)
test_df["price_latitude"] =  (test_df["price"])/ (test_df["latitude"]+1.0)

train_df["price_longtitude"] = (train_df["price"])/ (train_df["longitude"]-1.0)
test_df["price_longtitude"] =  (test_df["price"])/ (test_df["longitude"]-1.0)

In [16]:
features_to_use  = ['bathrooms', 'bedrooms', 'latitude', 'longitude', 'price']

In [17]:
features_to_use.extend(['num_photos', 
                        'num_features', 
                        'num_description_words',
                        'created_year', 
                        'created_month', 
                        'created_day', 
                        'listing_id', 
                        'created_hour', 
                        'clusters',
                        'distance_manh',
                        'distance_central_park',
                        'room_difference',
                        'total_rooms',
                        'price_per_room',
                         'bronx',
                        'distance_central_park',
                        'queens', 
                        'brooklin_cometery', 
                        'brooklin', 
                        'clusters',
                        'num_description_words', 
                        'created_year',
                        'created_month', 
                        'created_day', 
                        'created_hour', 
                        'room_difference',
                        'total_rooms', 
                        'subway_distance', 
                        "density", 
                        "price_latitude", 
                        "price_longtitude", 
                        'total_days', 
                        'diff_rank', 
                        'time_stamp', 
                        'price0', 
                        'price_per_bedroom', 
                        'price_per_bathroom'])

# FE Кодирование менеджеров

In [18]:
import random
start_values = [0,0,0]

index=list(range(train_df.shape[0]))
random.shuffle(index)
a=[np.nan]*len(train_df)
b=[np.nan]*len(train_df)
c=[np.nan]*len(train_df)

for i in range(5):
    building_level={}
    for j in train_df['manager_id'].values:
        building_level[j]= start_values.copy()
    test_index=index[int((i*train_df.shape[0])/5):int(((i+1)*train_df.shape[0])/5)]
    train_index=list(set(index).difference(test_index))
    for j in train_index:
        temp=train_df.iloc[j]
        if temp['interest_level']=='low':
            building_level[temp['manager_id']][0]+=1
        if temp['interest_level']=='medium':
            building_level[temp['manager_id']][1]+=1
        if temp['interest_level']=='high':
            building_level[temp['manager_id']][2]+=1
    for j in test_index:
        temp=train_df.iloc[j]
        if sum(building_level[temp['manager_id']])!=0:
            a[j]=building_level[temp['manager_id']][0]*1.0/sum(building_level[temp['manager_id']])
            b[j]=building_level[temp['manager_id']][1]*1.0/sum(building_level[temp['manager_id']])
            c[j]=building_level[temp['manager_id']][2]*1.0/sum(building_level[temp['manager_id']])
train_df['manager_level_low']=a
train_df['manager_level_medium']=b
train_df['manager_level_high']=c


a=[]
b=[]
c=[]
building_level={}
for j in train_df['manager_id'].values:
    building_level[j]= start_values.copy()
for j in range(train_df.shape[0]):
    temp=train_df.iloc[j]
    if temp['interest_level']=='low':
        building_level[temp['manager_id']][0]+=1
    if temp['interest_level']=='medium':
        building_level[temp['manager_id']][1]+=1
    if temp['interest_level']=='high':
        building_level[temp['manager_id']][2]+=1

for i in test_df['manager_id'].values:
    if i not in building_level.keys():
        a.append(np.nan)
        b.append(np.nan)
        c.append(np.nan)
    else:
        a.append(building_level[i][0]*1.0/sum(building_level[i]))
        b.append(building_level[i][1]*1.0/sum(building_level[i]))
        c.append(building_level[i][2]*1.0/sum(building_level[i]))
test_df['manager_level_low']=a
test_df['manager_level_medium']=b
test_df['manager_level_high']=c

features_to_use.append('manager_level_low') 
features_to_use.append('manager_level_medium') 
features_to_use.append('manager_level_high')

In [19]:
# Функция обучения
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=2017, num_rounds=2000):
    params = {
        'boosting_type': 'gbdt', 'objective': 'multiclass', 'nthread': -1, 'verbose': 0,
        'num_leaves': 54, 'learning_rate': 0.05, 'max_depth': -1,
        'num_class': 3, 'subsample': 0.8, 'subsample_freq': 1, 'colsample_bytree': 0.6, 
        'reg_alpha': 1, 'reg_lambda': 0.001, 'metric': 'multi_logloss',
        'min_split_gain': 0.5, 'min_child_weight': 1, 'min_child_samples': 10, 'scale_pos_weight': 1}
    
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed_val)
    for dev_index, val_index in kf.split(train_X, train_y):
        dev_X, val_X = train_X[dev_index,:], train_X[val_index,:]
        dev_y, val_y = train_y[dev_index], train_y[val_index]
        train_set = lgbm.Dataset(dev_X, dev_y, silent=True)
        val_set = lgbm.Dataset(val_X, val_y, silent=True)
        
        model = lgbm.train(params, train_set=train_set, num_boost_round=num_rounds, valid_sets=val_set,
                        early_stopping_rounds=50, evals_result=None, verbose_eval=100, learning_rates=None, 
                        callbacks=None)

    pred_test_y = model.predict(test_X, num_iteration = model.best_iteration)
    return pred_test_y, lgbm

In [20]:
# label encoding
categorical = ["display_address", "manager_id", "building_id"]
for f in categorical:
        if train_df[f].dtype=='object':
            lbl = LabelEncoder()
            lbl.fit(list(train_df[f].values) + list(test_df[f].values))
            train_df[f] = lbl.transform(list(train_df[f].values))
            test_df[f] = lbl.transform(list(test_df[f].values))
            features_to_use.append(f)

train_df['features'] = train_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
test_df['features'] = test_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))

tfidf = CountVectorizer(stop_words='english', max_features=200)
tr_sparse = tfidf.fit_transform(train_df["features"])
te_sparse = tfidf.transform(test_df["features"])

train_X = sparse.hstack([train_df[features_to_use], tr_sparse]).tocsr()
test_X = sparse.hstack([test_df[features_to_use], te_sparse]).tocsr()

target_num_map = {'high':0, 'medium':1, 'low':2}
train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))



start training


NameError: name 'StratifiedKFold' is not defined

In [22]:
print('start training')

preds, model = runXGB(train_X, train_y, test_X, num_rounds=2000)
out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_csv("lgb.csv", index=False)

start training
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006465 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 50 rounds
[100]	valid_0's multi_logloss: 0.53082
[200]	valid_0's multi_logloss: 0.521268
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[240]	valid_0's multi_logloss: 0.520094
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011043 seconds.
You can set `force_col_wise=true` to remove the overhead.
Training until validation scores don't improve for 50 rounds
[100]	valid_0's multi_logloss: 0.518461
[200]	valid_0's multi_logloss: 0.507398
[300]	valid_0's multi_logloss: 0.504929
[LightGBM] [Warning] No further splits with p

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[400]	valid_0's multi_logloss: 0.509473
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [23]:
out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_csv("lgbm.csv", index=False)